In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
wine_data = pd.read_csv('https://raw.githubusercontent.com/edyoda/data-science-complete-tutorial/master/Data/winequality-white.csv', sep=';')
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
wine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [4]:
# there are no missing values
# Quality is categorical and is our target column. We will check distribution
wine_data.quality.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [14]:
#there are very less quality 3 and 9. We might miss them during spliting data.
# we will distribute the quality column in 3 categories
def reduced_category(r):
    if r <= 3:
        return 1
    elif r<= 6:
        return 2
    else:
        return 3

wine_data.quality = wine_data.quality.map(reduced_category)

In [6]:
wine_data.quality.value_counts()

# category 1 is still sparce. We will use stratified Shuffle Split

2    3818
3    1060
1      20
Name: quality, dtype: int64

In [7]:
wine_data.corr()['quality']

#None of the features shows good correlation with quality

fixed acidity          -0.088004
volatile acidity       -0.071419
citric acid            -0.034883
residual sugar         -0.115124
chlorides              -0.183853
free sulfur dioxide    -0.033350
total sulfur dioxide   -0.166879
density                -0.281903
pH                      0.091992
sulphates               0.047926
alcohol                 0.380014
quality                 1.000000
Name: quality, dtype: float64

In [22]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(wine_data.drop(columns=['quality']), wine_data.quality)

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7983673469387755

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
lr= LogisticRegression()
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, n_jobs=2, random_state=42)

In [51]:
models = {"Logistic Regression":lr, "Decision tree": dt, "random forest": rf}

for name, model in models.items():
    my_pipeline = Pipeline([('std_scaler', StandardScaler()),
                          (name, model)])
    my_pipeline.fit(trainX, trainY)
    print("accuracy of ", name,":- ", my_pipeline.score(testX, testY))

accuracy of  Logistic Regression :-  0.5995111731843575
accuracy of  Decision tree :-  0.9490223463687151
accuracy of  random forest :-  0.9636871508379888


In [52]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

sampler = RandomOverSampler()

In [53]:
feature_s, target_s = sampler.fit_sample(wine_data.drop(columns=['quality']), wine_data['quality'])

trainX, testX, trainY, testY = train_test_split(feature_s, target_s)

for name, model in models.items():
    my_pipeline = Pipeline([('std_scaler', StandardScaler()),
                          (name, model)])
    my_pipeline.fit(trainX, trainY)
    print("accuracy of ", name,":- ", my_pipeline.score(testX, testY))

accuracy of  Logistic Regression :-  0.5921787709497207
accuracy of  Decision tree :-  0.9486731843575419
accuracy of  random forest :-  0.9706703910614525


In [31]:
# Random forest is now giving a good accuracy.
# We can use this model for further predictions